In [1]:
import os.path
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import requests

In [2]:
source = pd.read_csv("data/langmeta.csv").fillna(value = "")
stop_words_german = []
with open("data/stop_words_german.txt") as file:
    for line in file:
        stop_words_german.append(line.replace('\n', '').encode('ASCII','ignore').decode('ASCII'))

In [3]:
if (os.path.exists("data/keywordsmeta.csv")):
    os.remove("data/keywordsmeta.csv")

In [4]:

german_vec = CountVectorizer(
    ngram_range=(1, 1), 
    stop_words=stop_words_german,
    max_features=5,
    strip_accents='ascii'
)
english_vec = CountVectorizer(
    ngram_range=(1, 1), 
    stop_words="english",
    max_features=5,
    strip_accents='ascii'
)
#def process_text(text):
#    text = str(text).lower()
#    text = re.sub(
#        f"[{re.escape(string.punctuation)}]", " ", text
#    )
#    text = " ".join(text.split())
#    return text
#
#df["clean_text"] = pd.DataFrame.text.map(process_text)

In [5]:

text=[]
for index, row in source.iterrows():  
    metadict = {"Identifier":row["Identifier"]}      
    #pfad = "temp/" + row["Identifier"] + ".pdf"
    if(row['Language'] != ""):
        if(int(row['Textsize']) > 100):
            if(row['Language'] == "GERMAN"):
                textpfad = "temp/" + row["Identifier"] + ".txt"
                headertextpfad = "temp/" + row["Identifier"] + "_header.txt"
                with open(textpfad, 'r', encoding="utf-8") as textfile:
                    german_vec.fit_transform(textfile)
                    metadict.update({"Topwords":str(german_vec.get_feature_names_out())}) 
                try:
                    with open(headertextpfad, 'r', encoding="utf-8") as textfile:
                        german_vec.fit_transform(textfile)
                        metadict.update({"Header_Topwords":str(german_vec.get_feature_names_out())})
                except:
                    print("Error with Headertext")
            elif(row['Language'] == "ENGLISH"):
                textpfad = "temp/" + row["Identifier"] + ".txt"
                with open(textpfad, 'r', encoding="utf-8") as textfile:
                    english_vec.fit_transform(textfile)                    
                    metadict.update({"Topwords":str(english_vec.get_feature_names_out())}) 
                try:
                    with open(headertextpfad, 'r', encoding="utf-8") as textfile:
                        english_vec.fit_transform(textfile)                    
                        metadict.update({"Header_Topwords":str(english_vec.get_feature_names_out())}) 
                except:
                    print("Error with Headertext")
            else:
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/keywordsmeta.csv")):   
                    #oldframe = pd.read_csv("data/langmeta.csv")
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/keywordsmeta.csv", index=False)
                    oldframe = newframe
                else:
                    frame.to_csv("data/keywordsmeta.csv", index=False)
                    oldframe = frame
                metadict.clear
                topwords=""
                print("finished: ", row["Identifier"], " couldn't match language")
                continue
         #ask lobid.org to find a fitting keyword 
            if("Header_Topwords" in metadict):
                wordcloud = str(metadict["Header_Topwords"]).replace("'", "")\
                                                            .removeprefix("[")\
                                                            .removesuffix("]")\
                                                            .split(" ")
            else:
                wordcloud = str(metadict["Topwords"]).replace("'", "")\
                                                     .removeprefix("[")\
                                                     .removesuffix("]")\
                                                     .split(" ")
            for word in wordcloud:
                #/gnd/search?q=preferredName:Twain&filter=type:Person&format=json
                api_url = "https://lobid.org/gnd/search?q=preferredName:" + word + "&filter=type:SubjectHeading&format=json"
                print(api_url)
                response = requests.get(api_url)
                if(response.status_code == requests.codes.ok):
                    for member in response.json()["member"]: 
                        if('SubjectHeading' in member["type"] or 'SubjectHeadingSensoStricto' in member["type"]):
                            metadict.update({"KeywordID":member["id"]})
                            metadict.update({"Keywordname":member["preferredName"]})
                            foundstandard = True
                            break
                if(foundstandard):
                    break
                    
                    
                    
                    
                    
                    
                    
                    
        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/keywordsmeta.csv")):   
        #oldframe = pd.read_csv("data/langmeta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/keywordsmeta.csv", index=False)
            oldframe = newframe
        else:
            frame.to_csv("data/keywordsmeta.csv", index=False)
            oldframe = frame
    metadict.clear
    topwords=""
    foundstandard = False 
    print("finished: ", row["Identifier"])
    #break     


C:\Users\Johann\.virtualenvs\pipenv-5hQiS9Jl\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ernst', 'ordnung', 'schluss', 'wahr'] not in stop_words.
  warnings.warn(


https://lobid.org/gnd/search?q=preferredName:11&filter=type:SubjectHeading&format=json
finished:  8I6sM5zapD60
finished:  8ZICOHBmAHyQ
https://lobid.org/gnd/search?q=preferredName:bibliographie&filter=type:SubjectHeading&format=json
finished:  3ztCv-WpxJ4U
https://lobid.org/gnd/search?q=preferredName:begriffe&filter=type:SubjectHeading&format=json
finished:  6mOhjfscZK2A
https://lobid.org/gnd/search?q=preferredName:ereignisse&filter=type:SubjectHeading&format=json
finished:  1eteONeHL82Y
https://lobid.org/gnd/search?q=preferredName:audio&filter=type:SubjectHeading&format=json
finished:  1mjbqKfwSW7U
https://lobid.org/gnd/search?q=preferredName:abonnieren&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:benutzeroberflache&filter=type:SubjectHeading&format=json
finished:  1BruMQFjEIRY
https://lobid.org/gnd/search?q=preferredName:generics&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:nullable&filter=type:SubjectHeadi

finished:  1Xjd0hs1bOxE
https://lobid.org/gnd/search?q=preferredName:assemblies&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:assembly&filter=type:SubjectHeading&format=json
finished:  1cmen2XGcDe8
https://lobid.org/gnd/search?q=preferredName:02&filter=type:SubjectHeading&format=json
finished:  1yERWzyXd7r0
https://lobid.org/gnd/search?q=preferredName:de&filter=type:SubjectHeading&format=json
finished:  1mDaE1-TGeX4
https://lobid.org/gnd/search?q=preferredName:2018&filter=type:SubjectHeading&format=json
finished:  1pKFRKt4miHg
finished:  1GqJ1UWSoCLM  couldn't match language
https://lobid.org/gnd/search?q=preferredName:abbildung&filter=type:SubjectHeading&format=json
finished:  7XXSuW660wHA
https://lobid.org/gnd/search?q=preferredName:anmeldung&filter=type:SubjectHeading&format=json
finished:  7UynWfbOmtvY
finished:  4gYJdIg9_FDI
https://lobid.org/gnd/search?q=preferredName:ausgangslage&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/s

https://lobid.org/gnd/search?q=preferredName:empowerment&filter=type:SubjectHeading&format=json
finished:  9hz9f6wzzUk8
https://lobid.org/gnd/search?q=preferredName:erkenntnisse&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:kreuzungen&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:losung&filter=type:SubjectHeading&format=json
finished:  8394JVb2Sy_E
https://lobid.org/gnd/search?q=preferredName:2018&filter=type:SubjectHeading&format=json
finished:  6I-lYeH7oRIg
https://lobid.org/gnd/search?q=preferredName:bit&filter=type:SubjectHeading&format=json
finished:  3cMAp8O1PDjk
https://lobid.org/gnd/search?q=preferredName:01&filter=type:SubjectHeading&format=json
finished:  4Go34OQK9Xus
finished:  8NU3_-wc8J6U
https://lobid.org/gnd/search?q=preferredName:gezeichneten&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:kraft&filter=type:SubjectHeading&format=json
finished:  70yaDUVKkzO0
ht

https://lobid.org/gnd/search?q=preferredName:online&filter=type:SubjectHeading&format=json
finished:  9bo-jic3k6pY
https://lobid.org/gnd/search?q=preferredName:abtragen&filter=type:SubjectHeading&format=json
finished:  3wPrkJlM9uY8
https://lobid.org/gnd/search?q=preferredName:arduino&filter=type:SubjectHeading&format=json
finished:  3kcnKgnSAtpk
https://lobid.org/gnd/search?q=preferredName:afte&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:allgemeines&filter=type:SubjectHeading&format=json
finished:  4erg8BmO9Jso
https://lobid.org/gnd/search?q=preferredName:aufmerksamkeit&filter=type:SubjectHeading&format=json
finished:  4QV3zUKoIYt0
https://lobid.org/gnd/search?q=preferredName:15&filter=type:SubjectHeading&format=json
finished:  4yZKkpw4fNnA
https://lobid.org/gnd/search?q=preferredName:haufigkeit&filter=type:SubjectHeading&format=json
finished:  5Q0kdAEb2iqY
https://lobid.org/gnd/search?q=preferredName:ablaufplan&filter=type:SubjectHeading&format=

...

finished:  10L6IOoQjfnMY
https://lobid.org/gnd/search?q=preferredName:draw&filter=type:SubjectHeading&format=json
finished:  10wcBiIidwiYQ
https://lobid.org/gnd/search?q=preferredName:animationen&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:anwendung&filter=type:SubjectHeading&format=json
finished:  103h_L1r0wjOQ
https://lobid.org/gnd/search?q=preferredName:audioproduktion&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:einbinden&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:grundwissen&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:prasentationen
&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:vorbereitungen&filter=type:SubjectHeading&format=json
finished:  109_7OwqSc6eM
https://lobid.org/gnd/search?q=preferredName:alternativen&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=pref

https://lobid.org/gnd/search?q=preferredName:ubung&filter=type:SubjectHeading&format=json
finished:  10pyfgWQc9Gzk
https://lobid.org/gnd/search?q=preferredName:3d&filter=type:SubjectHeading&format=json
finished:  108xWlGul3UoA
https://lobid.org/gnd/search?q=preferredName:draw&filter=type:SubjectHeading&format=json
finished:  10ZEPc99l_yGk
https://lobid.org/gnd/search?q=preferredName:learning&filter=type:SubjectHeading&format=json
finished:  10FVro6fjMGGw
https://lobid.org/gnd/search?q=preferredName:add&filter=type:SubjectHeading&format=json
finished:  10HLDpcnPTMnA
https://lobid.org/gnd/search?q=preferredName:datensatze&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:ergebnissen&filter=type:SubjectHeading&format=json
finished:  106b_RitP6y4Y
https://lobid.org/gnd/search?q=preferredName:kommunikationskanale&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:lizenzierung&filter=type:SubjectHeading&format=json
https://lo

finished:  7frjMO4UK1u8
https://lobid.org/gnd/search?q=preferredName:feedback&filter=type:SubjectHeading&format=json
finished:  6Uyg-JgNSiFk
https://lobid.org/gnd/search?q=preferredName:2021&filter=type:SubjectHeading&format=json
finished:  6seY40ce3q_E
https://lobid.org/gnd/search?q=preferredName:00&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:2021&filter=type:SubjectHeading&format=json
finished:  6NazPdDiCkRA
https://lobid.org/gnd/search?q=preferredName:00&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:2021&filter=type:SubjectHeading&format=json
finished:  6nFM7ALYiJL4
https://lobid.org/gnd/search?q=preferredName:00&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:2021&filter=type:SubjectHeading&format=json
finished:  64xJVlDp0KDc
https://lobid.org/gnd/search?q=preferredName:00&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:2021&filter=ty

https://lobid.org/gnd/search?q=preferredName:auftraggeber&filter=type:SubjectHeading&format=json
finished:  7CqVPh1mqZKs
finished:  79XIkw_-0VHs
https://lobid.org/gnd/search?q=preferredName:auftragsklarung&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:ein&filter=type:SubjectHeading&format=json
finished:  5dCTkgT9Uowg
https://lobid.org/gnd/search?q=preferredName:algorithmierung&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:aufgabe&filter=type:SubjectHeading&format=json
finished:  7L8EI5xvFxgM
https://lobid.org/gnd/search?q=preferredName:dvgw&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:enwg&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:organisationsstruktur&filter=type:SubjectHeading&format=json
finished:  9WM-B7_W3Jyg
https://lobid.org/gnd/search?q=preferredName:00&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=p

https://lobid.org/gnd/search?q=preferredName:grafiken&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:karten&filter=type:SubjectHeading&format=json
finished:  9eywwe78hGWA
https://lobid.org/gnd/search?q=preferredName:exkurs&filter=type:SubjectHeading&format=json
finished:  8BdaJ9lQqryA
finished:  10_5HP1cwsYrw  couldn't match language
https://lobid.org/gnd/search?q=preferredName:asthetik&filter=type:SubjectHeading&format=json
finished:  8eGzsKQsCQCg
https://lobid.org/gnd/search?q=preferredName:bild&filter=type:SubjectHeading&format=json
finished:  12AjowN_VqTzk
https://lobid.org/gnd/search?q=preferredName:dorsalextension&filter=type:SubjectHeading&format=json
https://lobid.org/gnd/search?q=preferredName:grad&filter=type:SubjectHeading&format=json
finished:  12va0-q8Oqi6k
https://lobid.org/gnd/search?q=preferredName:asl&filter=type:SubjectHeading&format=json
finished:  12k7oTJwEVuh4
https://lobid.org/gnd/search?q=preferredName:brenngasen&filter=type:S

In [6]:
teststr = ["Das sind ganz besondere Testlisten. Der Hammer und Zäune und Hühner", 
          "Testlisten und Hammer",
          "Hammer, Testlisten und Zäune",
          "Ansonsten auf Bänke und Tische gibts Hühner auf den Testlisten"]
german_vec.fit_transform(teststr)
german_vec.get_feature_names_out()

array(['ansonsten', 'hammer', 'huhner', 'testlisten', 'zaune'],
      dtype=object)

In [7]:
#strange =
for word in str(metadict["Topwords"]).replace("'", "")\
                                     .removeprefix("[")\
                                     .removesuffix("]")\
                                     .split(" "):
    print(word)

2021
aerodynamik
innen
rohr
schuler


In [8]:
"Header_Topwords" in metadict

True